In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# Step 1: Set up directories and preprocess data
train_dir = 'path_to_food-101/train'  # Replace with the actual path
test_dir = 'path_to_food-101/test'    # Replace with the actual path

# Data augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Only rescale the test data
test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators for training and validation sets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

# Step 2: Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(101, activation='softmax')  # 101 food classes
])

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Print the model architecture
model.summary()

# Step 3: Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator)

# Step 4: Visualize training and validation accuracy
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# Plot training and validation accuracy
plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

# Plot training and validation loss
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Step 5: Calorie lookup table
calorie_dict = {
    'apple_pie': 320,
    'pizza': 266,
    'spaghetti_bolognese': 450,
    'hamburger': 295,
    'sushi': 200,
    # Add more food items with their corresponding calorie values
}

def estimate_calories(food_item):
    # Return the calorie value from the dictionary or indicate it's not available
    return calorie_dict.get(food_item, "Calorie data not available")

# Step 6: Function to predict food and estimate calories
def predict_food_and_calories(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(150, 150))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction[0])

    # Get the food item name from the class index
    food_item = list(train_generator.class_indices.keys())[predicted_class_index]

    # Estimate the calories for the predicted food item
    calories = estimate_calories(food_item)

    # Output the results
    print(f'Predicted food item: {food_item}')
    print(f'Estimated calories: {calories}')

# Step 7: Test the model with a sample image
img_path = 'path_to_test_image.jpg'  # Replace with the actual path to your image
predict_food_and_calories(img_path)
